# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9ee0e59d90>
├── 'a' --> tensor([[-0.5805, -0.9187,  1.5745],
│                   [-0.9200,  0.1780,  1.1750]])
└── 'x' --> <FastTreeValue 0x7f9ee0e592b0>
    └── 'c' --> tensor([[ 1.4841, -0.4748, -0.1829,  1.4843],
                        [-0.1868, -0.8543,  0.5927, -1.3745],
                        [-0.4128,  0.5505,  0.2066,  0.2966]])

In [4]:
t.a

tensor([[-0.5805, -0.9187,  1.5745],
        [-0.9200,  0.1780,  1.1750]])

In [5]:
%timeit t.a

62 ns ± 0.0853 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9ee0e59d90>
├── 'a' --> tensor([[0.5596, 0.6354, 0.3085],
│                   [1.0607, 0.3590, 0.1221]])
└── 'x' --> <FastTreeValue 0x7f9ee0e592b0>
    └── 'c' --> tensor([[ 1.4841, -0.4748, -0.1829,  1.4843],
                        [-0.1868, -0.8543,  0.5927, -1.3745],
                        [-0.4128,  0.5505,  0.2066,  0.2966]])

In [7]:
%timeit t.a = new_value

61.8 ns ± 0.0454 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5805, -0.9187,  1.5745],
               [-0.9200,  0.1780,  1.1750]]),
    x: Batch(
           c: tensor([[ 1.4841, -0.4748, -0.1829,  1.4843],
                      [-0.1868, -0.8543,  0.5927, -1.3745],
                      [-0.4128,  0.5505,  0.2066,  0.2966]]),
       ),
)

In [10]:
b.a

tensor([[-0.5805, -0.9187,  1.5745],
        [-0.9200,  0.1780,  1.1750]])

In [11]:
%timeit b.a

54.1 ns ± 0.156 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0202, -1.2519, -1.0017],
               [-0.1771, -0.6391, -0.4135]]),
    x: Batch(
           c: tensor([[ 1.4841, -0.4748, -0.1829,  1.4843],
                      [-0.1868, -0.8543,  0.5927, -1.3745],
                      [-0.4128,  0.5505,  0.2066,  0.2966]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.15 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

841 ns ± 0.237 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 53.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 1.15 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 2.22 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9e36afa670>
├── 'a' --> tensor([[[-0.5805, -0.9187,  1.5745],
│                    [-0.9200,  0.1780,  1.1750]],
│           
│                   [[-0.5805, -0.9187,  1.5745],
│                    [-0.9200,  0.1780,  1.1750]],
│           
│                   [[-0.5805, -0.9187,  1.5745],
│                    [-0.9200,  0.1780,  1.1750]],
│           
│                   [[-0.5805, -0.9187,  1.5745],
│                    [-0.9200,  0.1780,  1.1750]],
│           
│                   [[-0.5805, -0.9187,  1.5745],
│                    [-0.9200,  0.1780,  1.1750]],
│           
│                   [[-0.5805, -0.9187,  1.5745],
│                    [-0.9200,  0.1780,  1.1750]],
│           
│                   [[-0.5805, -0.9187,  1.5745],
│                    [-0.9200,  0.1780,  1.1750]],
│           
│                   [[-0.5805, -0.9187,  1.5745],
│                    [-0.9200,  0.1780,  1.1750]]])
└── 'x' --> <FastTreeValue 0x7f9e36afa4f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 43.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9e3721bca0>
├── 'a' --> tensor([[-0.5805, -0.9187,  1.5745],
│                   [-0.9200,  0.1780,  1.1750],
│                   [-0.5805, -0.9187,  1.5745],
│                   [-0.9200,  0.1780,  1.1750],
│                   [-0.5805, -0.9187,  1.5745],
│                   [-0.9200,  0.1780,  1.1750],
│                   [-0.5805, -0.9187,  1.5745],
│                   [-0.9200,  0.1780,  1.1750],
│                   [-0.5805, -0.9187,  1.5745],
│                   [-0.9200,  0.1780,  1.1750],
│                   [-0.5805, -0.9187,  1.5745],
│                   [-0.9200,  0.1780,  1.1750],
│                   [-0.5805, -0.9187,  1.5745],
│                   [-0.9200,  0.1780,  1.1750],
│                   [-0.5805, -0.9187,  1.5745],
│                   [-0.9200,  0.1780,  1.1750]])
└── 'x' --> <FastTreeValue 0x7f9e3721bc10>
    └── 'c' --> tensor([[ 1.4841, -0.4748, -0.1829,  1.4843],
                        [-0.1868, -0.8543,  0.5927, -1.3745],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 18.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.5 µs ± 81.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.5805, -0.9187,  1.5745],
                [-0.9200,  0.1780,  1.1750]],
       
               [[-0.5805, -0.9187,  1.5745],
                [-0.9200,  0.1780,  1.1750]],
       
               [[-0.5805, -0.9187,  1.5745],
                [-0.9200,  0.1780,  1.1750]],
       
               [[-0.5805, -0.9187,  1.5745],
                [-0.9200,  0.1780,  1.1750]],
       
               [[-0.5805, -0.9187,  1.5745],
                [-0.9200,  0.1780,  1.1750]],
       
               [[-0.5805, -0.9187,  1.5745],
                [-0.9200,  0.1780,  1.1750]],
       
               [[-0.5805, -0.9187,  1.5745],
                [-0.9200,  0.1780,  1.1750]],
       
               [[-0.5805, -0.9187,  1.5745],
                [-0.9200,  0.1780,  1.1750]]]),
    x: Batch(
           c: tensor([[[ 1.4841, -0.4748, -0.1829,  1.4843],
                       [-0.1868, -0.8543,  0.5927, -1.3745],
                       [-0.4128,  0.5505,  0.2066,  0.2966]],
         

In [26]:
%timeit Batch.stack(batches)

87.4 µs ± 86.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.5805, -0.9187,  1.5745],
               [-0.9200,  0.1780,  1.1750],
               [-0.5805, -0.9187,  1.5745],
               [-0.9200,  0.1780,  1.1750],
               [-0.5805, -0.9187,  1.5745],
               [-0.9200,  0.1780,  1.1750],
               [-0.5805, -0.9187,  1.5745],
               [-0.9200,  0.1780,  1.1750],
               [-0.5805, -0.9187,  1.5745],
               [-0.9200,  0.1780,  1.1750],
               [-0.5805, -0.9187,  1.5745],
               [-0.9200,  0.1780,  1.1750],
               [-0.5805, -0.9187,  1.5745],
               [-0.9200,  0.1780,  1.1750],
               [-0.5805, -0.9187,  1.5745],
               [-0.9200,  0.1780,  1.1750]]),
    x: Batch(
           c: tensor([[ 1.4841, -0.4748, -0.1829,  1.4843],
                      [-0.1868, -0.8543,  0.5927, -1.3745],
                      [-0.4128,  0.5505,  0.2066,  0.2966],
                      [ 1.4841, -0.4748, -0.1829,  1.4843],
                      [-0.1868, -

In [28]:
%timeit Batch.cat(batches)

160 µs ± 445 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
